# T07: Transformación de Datos a RDF (Grafo de Conocimiento)

Este notebook realiza el proceso de **ETL (Extract, Transform, Load)** para convertir los datos abiertos de los distritos de Valencia en un **Grafo de Conocimiento** enriquecido.

### Objetivos:
1.  **Cargar** los datos de distritos desde un archivo CSV.
2.  **Transformar** los datos al modelo RDF utilizando el vocabulario **schema.org**.
3.  **Enriquecer** los datos enlazándolos con **Wikidata** (Linked Data).
4.  **Generar** un archivo `.ttl` (Turtle) listo para ser consultado.

In [1]:
# 1. Importación de Librerías
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, DCTERMS, DC, SKOS, XSD, OWL
import pandas as pd
import os   
import unicodedata
import re

# Definición del Dominio Base para nuestros recursos
domain = 'https://valencia.example.org/' 
print("Librerías importadas y dominio configurado.")

Librerías importadas y dominio configurado.


## 2. Funciones Auxiliares y Mapeo
Definimos funciones para normalizar texto y crear URIs válidas. Además, establecemos un **diccionario de mapeo** manual para conectar nuestros distritos con sus identificadores únicos en **Wikidata** (QIDs).

In [2]:
# --- FUNCIONES AUXILIARES ---

# Función para crear slugs seguros (URIs limpias)
def slugify(text):
    text = unicodedata.normalize("NFD", text)
    text = text.encode("ascii", "ignore").decode("utf-8")
    text = re.sub(r"[^a-zA-Z0-9\-]", "-", text)
    text = re.sub(r"-+", "-", text)
    return text.lower().strip("-")

# Función de normalización para coincidencia robusta
def normalize_for_match(text):
    if not isinstance(text, str): return ""
    text = unicodedata.normalize("NFD", text)
    text = text.encode("ascii", "ignore").decode("utf-8")
    return text.upper().strip()

# --- MAPEO EXPLÍCITO: Diccionario manual de Wikidata ---
WIKIDATA_MAPPING = {
    "CIUTAT VELLA": "Q3392733", 
    "L'EIXAMPLE": "Q3392791",   
    "EXTRAMURS": "Q3393056",      
    "CAMPANAR": "Q3393032",     
    "LA SAIDIA": "Q3393007",   
    "EL PLA DEL REAL": "Q4452090",
    "L'OLIVERETA": "Q3392792",  
    "PATRAIX": "Q55226",      
    "JESUS": "Q3393002",        
    "QUATRE CARRERES": "Q3393088",
    "POBLATS MARITIMS": "Q3392780",
    "CAMINS AL GRAU": "Q3392782",
    "ALGIROS": "Q3392835",     
    "BENIMACLET": "Q536707",   
    "RASCANYA": "Q3392759",     
    "BENICALAP": "Q777986",   
    "POBLATS DEL NORD": "Q3393120",
    "POBLATS DE L'OEST": "Q3393115",
    "POBLATS DEL SUD": "Q3392798",  
}
# Normalizamos las claves
WIKIDATA_MAPPING = {normalize_for_match(k): v for k, v in WIKIDATA_MAPPING.items()}
print("Funciones y mapeo Wikidata listos.")

Funciones y mapeo Wikidata listos.


## 3. Inicialización del Grafo RDF
Creamos un grafo vacío y definimos los **Namespaces** (vocabularios) que vamos a utilizar.
*   **schema**: Vocabulario principal para describir lugares.
*   **owl/skos**: Para relaciones semánticas y jerarquías.
*   **geo**: Para datos geoespaciales.

In [3]:
# Instancia el grafo y los namespaces
g = Graph()
g.bind("rdf", RDF)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
g.bind("skos", SKOS)
g.bind("xsd", XSD)
g.bind("owl", OWL)

# Namespace principal schema.org
schema = Namespace("https://schema.org/")
g.bind("schema", schema)

# Namespace para GeoSPARQL
GEO = Namespace("http://www.opengis.net/ont/geosparql#")
g.bind("geo", GEO)

print("Grafo inicializado con Namespaces.")

Grafo inicializado con Namespaces.


## 4. Carga de Datos (Extract)
Leemos el archivo CSV original que contiene la información de los distritos.

In [4]:
print("--- Cargando y procesando: districtes-distritos.csv ---")

# Ajuste de rutas
csv_path = '../data/districtes-distritos.csv'
if not os.path.exists(csv_path):
    csv_path = 'data/districtes-distritos.csv'

try:
    df_distritos = pd.read_csv (csv_path, sep=';')
    print(f"CSV cargado con éxito desde {csv_path}. Total de filas: {len(df_distritos)}")
except FileNotFoundError:
    print(f"Error: No se encuentra el archivo en {csv_path}")

--- Cargando y procesando: districtes-distritos.csv ---
CSV cargado con éxito desde ../data/districtes-distritos.csv. Total de filas: 19


## 5. Transformación a RDF (Transform)
Iteramos sobre cada fila del CSV para crear recursos RDF.
Para cada distrito:
1.  Creamos una **URI única** (ej: `.../district/ciutat-vella`).
2.  Asignamos el tipo `schema:Place`.
3.  Añadimos propiedades: nombre, identificador.
4.  **Enriquecemos** con el enlace a Wikidata (`sameAs`).
5.  Añadimos información geográfica (polígono y coordenadas).

In [5]:
count_processed = 0

print("Iniciando transformación de datos...")

for index, row in df_distritos.iterrows():
    # Sanitiza el nombre y código
    nombre_distrito = row["Nombre"].strip()
    nombre_key = normalize_for_match(nombre_distrito)
    
    # Filtrar filas inválidas
    if nombre_key not in WIKIDATA_MAPPING:
        continue
    
    count_processed += 1
    codigo_distrito = str(row["Código distrito"]).strip()
    
    # URI del recurso
    place_uri_name = slugify(nombre_distrito)
    place_uri = URIRef(domain + 'district/' + place_uri_name)
    
    # 1. Definición del Tipo (schema:Place)
    g.add((place_uri, RDF.type, schema.Place))
    g.add((place_uri, RDF.type, SKOS.Concept))
    
    # 2. Propiedades Básicas
    g.add((place_uri, schema.name, Literal(nombre_distrito, lang="ca")))
    g.add((place_uri, SKOS.prefLabel, Literal(nombre_distrito, lang="ca")))
    g.add((place_uri, schema.identifier, Literal(codigo_distrito)))

    # 3. Enlace con Wikidata (sameAs)
    qid = WIKIDATA_MAPPING.get(nombre_key)
    if qid:
        wikidata_uri = URIRef(f"http://www.wikidata.org/entity/{qid}")
        g.add((place_uri, schema.sameAs, wikidata_uri)) 
        g.add((place_uri, OWL.sameAs, wikidata_uri))

    # 4. Geometría: Polígono (GeoShape)
    if pd.notnull(row["geo_shape"]):
        shape_uri = URIRef(place_uri + "/geoshape")
        g.add((place_uri, schema.geo, shape_uri))
        g.add((shape_uri, RDF.type, schema.GeoShape))
        g.add((shape_uri, schema.polygon, Literal(row["geo_shape"]))) 

    # 5. Geometría: Punto Central (GeoCoordinates)
    if pd.notnull(row["geo_point_2d"]):
        lat_lon = row["geo_point_2d"].split(',')
        lat = lat_lon[0].strip()
        lon = lat_lon[1].strip()
        
        geo_coord = URIRef(place_uri + "/center")
        g.add((place_uri, schema.geo, geo_coord)) 
        g.add((geo_coord, RDF.type, schema.GeoCoordinates))
        g.add((geo_coord, schema.latitude, Literal(lat, datatype=XSD.decimal)))
        g.add((geo_coord, schema.longitude, Literal(lon, datatype=XSD.decimal)))

print(f"Transformación completada. Procesados {count_processed} distritos.")

Iniciando transformación de datos...
Transformación completada. Procesados 19 distritos.


## 6. Serialización y Verificación (Load)
Guardamos el grafo resultante en un archivo `.ttl` (Turtle) y realizamos una comprobación automática para asegurar que se han procesado todos los registros correctamente.

In [6]:
# Guardamos el resultado en formato Turtle (ttl)
output_dir = "../data/rdf"
if not os.path.exists("../data"):
    output_dir = "data/rdf"

output_file_name = "valencia_districts_places_enriched.ttl"
output_file = os.path.join(output_dir, output_file_name)

os.makedirs(output_dir, exist_ok=True) 
g.serialize(destination=output_file)

print(f"Grafo RDF guardado en: {output_file}")
print(f"Total de triples: {len(g)}")

# --- VERIFICACIÓN ---
num_places = len(list(g.subjects(RDF.type, schema.Place)))
if num_places == count_processed:
    print(f"✅ ÉXITO: Se generaron {num_places} recursos schema:Place (coincide con filas procesadas).")
else:
    print(f"⚠️ ALERTA: Generados {num_places} recursos vs {count_processed} filas procesadas.")

# Ejemplo de visualización
print("\nEjemplo de Triples (Ciutat Vella):")
example_uri = URIRef(domain + 'district/ciutat-vella')
for s, p, o in g.triples((example_uri, None, None)):
    print(f"  {p.split('/')[-1]} -> {o}")

Grafo RDF guardado en: ../data/rdf\valencia_districts_places_enriched.ttl
Total de triples: 266
✅ ÉXITO: Se generaron 19 recursos schema:Place (coincide con filas procesadas).

Ejemplo de Triples (Ciutat Vella):
  22-rdf-syntax-ns#type -> https://schema.org/Place
  22-rdf-syntax-ns#type -> http://www.w3.org/2004/02/skos/core#Concept
  name -> CIUTAT VELLA
  core#prefLabel -> CIUTAT VELLA
  identifier -> 1
  sameAs -> http://www.wikidata.org/entity/Q3392733
  owl#sameAs -> http://www.wikidata.org/entity/Q3392733
  geo -> https://valencia.example.org/district/ciutat-vella/geoshape
  geo -> https://valencia.example.org/district/ciutat-vella/center


## 7. Visualización (SPARQL)

Para verificar que los enlaces con Wikidata son correctos, puedes ejecutar la siguiente consulta **SPARQL** en el [Servicio de Consultas de Wikidata](https://query.wikidata.org/).

Esta consulta:
1.  Selecciona los distritos que hemos enlazado.
2.  Obtiene sus coordenadas desde Wikidata.
3.  Los muestra en un mapa interactivo.

Copia y pega el siguiente código en el editor de Wikidata:

In [ ]:
#defaultView:Map
SELECT ?r ?rLabel ?location
WHERE {
  # Lista de distritos de Valencia (QIDs)
  VALUES ?r { 
    wd:Q3392835 wd:Q777986 wd:Q536707 wd:Q3392782 wd:Q3393032 
    wd:Q3392733 wd:Q4452090 wd:Q3393056 wd:Q3393002 wd:Q3392791 
    wd:Q3392792 wd:Q3393007 wd:Q55226 wd:Q3393115 wd:Q3393120 
    wd:Q3392798 wd:Q3392780 wd:Q3393088 wd:Q3392759 
  }
  
  ?r wdt:P625 ?location. 
  
  SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],es". 
    ?r rdfs:label ?rLabel. 
  }
}